In [1]:
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

In [2]:
dfoff=pd.read_csv('train_offline.csv')
dftest=pd.read_csv('test_offline.csv')
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)
dfoff.head(10)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN


In [3]:
## Creat target label
def label(row):
    if(np.isnan(row['Date_received'])):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format
        ='%Y%m%d') - pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td<= pd.Timedelta(15, 'D'):
            return 1
    return 0
dfoff['label'] = dfoff.apply(label, axis=1)
dfoff.reset_index(drop=True, inplace=True)
dfoff.to_csv("train_offline_label.csv", index_label=False )
dfoff['label'].value_counts()
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0


In [4]:
dfoff = pd.read_csv('train_offline_label.csv')
dftest = pd.read_csv('test_offline.csv')
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=False)
print(dfoff.shape)
print(dftest.shape)
dfoff.head(10)

(1160742, 8)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0,-1
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN,0
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0


In [5]:
# Generate features - weekday acquired coupon
def getWeekday(row):# add one to make it from 0~6 -> 1~7
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1
    
dfoff['weekday']=dfoff['Date_received'].apply(getWeekday)
dftest['weekday']=dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [6]:
weekdaycols = ['weekday_'+str(i) for i in range(1,8)]
print(weekdaycols)

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [7]:
tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

In [8]:
# Generate features - coupon discount and distance
def getDiscountType(row):
    if (row == 'null'):
        return 'null'
    elif(':' in row):
        return 1
    else:
        return 0
    
#convert discount to rate
def convertRate(row):
    if(row=='null'):
        return 1.0
    elif(':' in row):
        rows = row.split(':')
        return 1.0 - ((float)(rows[1])/(float)(rows[0]))
    else:
        return (float)(row)
    
def getDiscountMan(row):
    if (':' in row):
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if (':' in row):
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0
    
def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df
dfoff = dfoff[dfoff['label'] != -1].copy()
dfoff = processData(dfoff)
dftest = processData(dftest)
dftest.shape

(306313, 19)

In [9]:
#convert distance
dfoff['distance'] = dfoff['Distance'].replace('null',-1).astype(int)
print(dfoff['distance'].unique())
dftest['distance'] = dftest['Distance'].replace('null',-1).astype(int)
print(dftest['distance'].unique())
dfoff.head(10)

[ 0  1  2 99 10  4  7  9  3  5  6  8]
[ 1  0 99  7  2 10  9  4  8  3  6  5]


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type,distance
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,1,0,0,0,0,0.95,20,1,1,0
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,...,0,0,0,1,0,0.95,20,1,1,0
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5.0,0,...,0,0,1,0,0,0.90,200,20,1,0
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,...,0,0,1,0,0,0.90,200,20,1,1
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,0,...,0,0,1,0,0,0.50,10,5,1,2
6,73611,2099,12034.0,100:10,99.0,20160207.0,NaN,0,7.0,0,...,0,0,0,0,1,0.90,100,10,1,99
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN,0,4.0,0,...,0,1,0,0,0,0.85,200,30,1,10
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN,0,6.0,0,...,0,0,0,1,0,0.90,200,20,1,10
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN,0,2.0,0,...,0,0,0,0,0,0.90,200,20,1,2
11,253750,8390,7531.0,20:5,0.0,20160327.0,NaN,0,7.0,0,...,0,0,0,0,1,0.75,20,5,1,0


In [10]:
dfoff['distance']

1           0
2           0
3           0
4           1
5           2
6          99
7          10
8          10
9           2
11          0
12          2
14         10
16          2
17          1
18          0
19          1
21         10
22         10
23         10
24         10
26          0
27          0
28          0
29          0
30          0
31         99
32         10
33          4
34         99
35          0
           ..
1160698     4
1160699    10
1160701     0
1160703     1
1160704     1
1160705     0
1160706     0
1160707     1
1160709     4
1160710     4
1160711     2
1160713     2
1160715    99
1160717     0
1160718     7
1160719     1
1160720     1
1160721     1
1160722     4
1160723     1
1160724     0
1160725     0
1160726     7
1160727     0
1160728     1
1160732     0
1160734     0
1160737     2
1160738     1
1160739     2
Name: distance, Length: 746969, dtype: int32

In [11]:
t1=dfoff[['User_id']]
t1['this_month_user_receive_all_coupon_count']=1
t1 = t1.groupby('User_id').agg('sum').reset_index()
#將t1按照user_id進行分組，然後統計所有用戶收取的優惠券數目，並初始化一個索引值

t2=dfoff[['User_id','Coupon_id']]
t2['this_month_user_receive_same_coupon_count'] = 1
t2 = t2.groupby(['User_id','Coupon_id']).agg('sum').reset_index()
#提取這個月用戶收到的相同優惠券的數目

t3=dfoff[['User_id','Coupon_id','Date_received']]
t3['this_month_use']=1
t3=t3.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()
#優惠券接收時間

t4 = dfoff[['User_id','Date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()
#用戶收到的優惠券量

#t5 = dfoff[['User_id','Merchant_id']]
#t5['this_merchant_get']=1
#t5 = t5.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
#提取用戶和商品數量

dfoff = pd.merge(dfoff,t1,on='User_id',how='left')
dfoff = pd.merge(dfoff,t2,on=['User_id','Coupon_id'],how='left')
dfoff = pd.merge(dfoff,t3,on=['User_id','Coupon_id','Date_received'],how='left')
dfoff = pd.merge(dfoff,t4,on=['User_id','Date_received'],how='left')
#dfoff = pd.merge(dfoff,t5,on=['User_id','Merchant_id'],how='left')
dfoff.columns

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'Date', 'label', 'weekday', 'weekday_type',
       'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5',
       'weekday_6', 'weekday_7', 'discount_rate', 'discount_man',
       'discount_jian', 'discount_type', 'distance',
       'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_count', 'this_month_use',
       'this_day_user_receive_all_coupon_count', 'this_merchant_get'],
      dtype='object')

In [12]:
t1=dftest[['User_id']]
t1['this_month_user_receive_all_coupon_count']=1
t1 = t1.groupby('User_id').agg('sum').reset_index()
#將t1按照user_id進行分組，然後統計所有用戶收取的優惠券數目，並初始化一個索引值

t2=dftest[['User_id','Coupon_id']]
t2['this_month_user_receive_same_coupon_count'] = 1
t2 = t2.groupby(['User_id','Coupon_id']).agg('sum').reset_index()
#提取這個月用戶收到的相同優惠券的數目

t3=dftest[['User_id','Coupon_id','Date_received']]
t3['this_month_use']=1
t3=t3.groupby(['User_id','Coupon_id','Date_received']).agg('sum').reset_index()
#優惠券接收時間

t4 = dftest[['User_id','Date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['User_id','Date_received']).agg('sum').reset_index()
#用戶收到的優惠券量

#t5 = dftest[['User_id','Merchant_id']]
#t5['this_merchant_get']=1
#t5 = t5.groupby(['User_id','Merchant_id']).agg('sum').reset_index()
#提取用戶和商品數量

dftest = pd.merge(dftest,t1,on='User_id',how='left')
dftest = pd.merge(dftest,t2,on=['User_id','Coupon_id'],how='left')
dftest = pd.merge(dftest,t3,on=['User_id','Coupon_id','Date_received'],how='left')
dftest = pd.merge(dftest,t4,on=['User_id','Date_received'],how='left')
#dftest = pd.merge(dftest,t5,on=['User_id','Merchant_id'],how='left')
dftest.columns

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

Index(['User_id', 'Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance',
       'Date_received', 'weekday', 'weekday_type', 'weekday_1', 'weekday_2',
       'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7',
       'discount_rate', 'discount_man', 'discount_jian', 'discount_type',
       'distance', 'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_count', 'this_month_use',
       'this_day_user_receive_all_coupon_count', 'this_merchant_get'],
      dtype='object')

In [13]:
#Navie model
def split_train_valid(row, date_cut="20160416"):
    is_train = True if pd.to_datetime(row, format="%Y%m%d") < pd.to_datetime(date_cut, format="%Y%m%d") else False
    return is_train
df = dfoff[dfoff['label'] != -1].copy()
df["is_train"] = df["Date_received"].apply(split_train_valid)
train = df[df["is_train"]]
valid = df[~df["is_train"]]
train.reset_index(drop=True, inplace=True)
train.to_csv("train.csv", index_label=False ) 
valid.reset_index(drop=True, inplace=True)
valid.to_csv("valid.csv", index_label=False ) 
print("Train size: {}, #positive: {}".format(len(train), train["label"].sum()))
print("Valid size: {}, #positive: {}".format(len(valid), valid["label"].sum()))

Train size: 667753, #positive: 32472
Valid size: 79216, #positive: 3832


In [14]:
original_feature = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'weekday_type','distance',
                   'this_month_user_receive_all_coupon_count',
       'this_month_user_receive_same_coupon_count', 'this_month_use',
       'this_day_user_receive_all_coupon_count'] + weekdaycols
predictors = original_feature
print(len(original_feature),original_feature)

19 ['discount_rate', 'discount_type', 'discount_man', 'discount_jian', 'Distance', 'weekday', 'weekday_type', 'distance', 'this_month_user_receive_all_coupon_count', 'this_month_user_receive_same_coupon_count', 'this_month_use', 'this_day_user_receive_all_coupon_count', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [15]:
train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,discount_man,discount_jian,discount_type,distance,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_count,this_month_use,this_day_user_receive_all_coupon_count,this_merchant_get,is_train
0,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3.0,0,...,20,1,1,0,2,1,1,1,2,True
1,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6.0,0,...,20,1,1,0,2,1,1,1,2,True
2,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5.0,0,...,200,20,1,1,1,1,1,1,1,True
3,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5.0,0,...,10,5,1,2,1,1,1,1,1,True
4,73611,2099,12034.0,100:10,99.0,20160207.0,NaN,0,7.0,0,...,100,10,1,99,1,1,1,1,1,True


In [16]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import roc_auc_score, accuracy_score

In [17]:
model_rfc = RandomForestClassifier(n_estimators=20, max_depth=4)
# 訓練模型
model_rfc.fit(train[predictors], train['label'])

# 預測測試集
pred = model_rfc.predict(valid[predictors])

# 評分
auc_score = roc_auc_score(y_true=valid.label, y_score=pred)
acc = accuracy_score(y_true=valid.label, y_pred=pred)

print("Validation AUC: {:.3f}, Accuracy: {:.3f}".format(auc_score, acc))

Validation AUC: 0.513, Accuracy: 0.953


In [18]:
def exportSubmit(model, filename):

    targetset = dftest.copy()
    print(targetset.shape)

    targetset = targetset[~targetset.Coupon_id.isna()]
    targetset.reset_index(drop=True, inplace=True)
    testset = targetset[predictors].copy()

    y_test_pred = model.predict_proba(testset[predictors])
    test1 = testset.copy()
    test1['pred_prob'] = y_test_pred[:, 1]
    print(test1.shape)
    
    output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
    print(output.shape)

    output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
    output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
    output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
    output.reset_index(drop=True, inplace=True)
    
    ### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
    out = output.groupby("uid", as_index=False).mean()
    out = out[["uid", "pred_prob"]]
    out.columns = ["uid", "label"]
    out.to_csv(filename, header=["uid", "label"], index=False) # submission format
    out.head()

In [19]:
exportSubmit(model_rfc, "rf_result.csv")

(306313, 25)
(306313, 20)
(306313, 4)


In [20]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [21]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train[predictors], train['label'],
                                                    test_size=0.25, random_state=42)

# 建立模型
clf = GradientBoostingClassifier()

In [22]:
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

0.9532164443299649


In [23]:
# 設定要訓練的超參數組合
n_estimators = [50, 100, 150]
max_depth = [1, 3, 5]
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


C:\Users\user\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 13.1min finished


In [24]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.953693 using {'max_depth': 5, 'n_estimators': 150}


In [25]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [26]:
# 調整參數後約可提高至 0.972 的準確率
print(metrics.accuracy_score(y_test, y_pred))

0.953725612349421


In [27]:
exportSubmit(clf_bestparam, "GBC_result.csv")

(306313, 25)
(306313, 20)
(306313, 4)
